In [19]:
import os
import pandas as pd 
from collections import deque
import random 

In [13]:
merged_df = pd.DataFrame()

In [14]:
crytos = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]

## DATASET PREPROCESSING

In [15]:
for crypto in crytos:
    print(f"Processing {crypto}")
    dataset_path = f"data/{crypto}.csv"
    
    df = pd.read_csv(dataset_path, names=['time', 'low', 'high', 'open', 'close', 'volume'])
    print(df.head())
    df.rename(columns={"close" : f"{crypto}_close", "volume":f"{crypto}_volume"}, inplace=True)
    
    df.set_index("time", inplace=True)    
    df=df[[f"{crypto}_close", f"{crypto}_volume"]]
    
    if len(merged_df)==0:
        merged_df = df
    else:
        merged_df = merged_df.join(df)
        
merged_df.fillna(method='ffill', inplace=True)
merged_df.dropna(inplace=True)



Processing BTC-USD
         time          low         high         open        close    volume
0  1528968660  6489.549805  6489.560059  6489.560059  6489.549805  0.587100
1  1528968720  6487.370117  6489.560059  6489.549805  6487.379883  7.706374
2  1528968780  6479.410156  6487.370117  6487.370117  6479.410156  3.088252
3  1528968840  6479.410156  6479.419922  6479.419922  6479.410156  1.404100
4  1528968900  6475.930176  6479.979980  6479.410156  6479.979980  0.753000
Processing LTC-USD
         time        low       high       open      close      volume
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067
4  1528968900  96.279999  96.540001  96.500000  96.389999  524.539978
Processing BCH-USD
         time         low        high        open       close     v

/tmp/ipykernel_34930/3352571941.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method='ffill', inplace=True)


In [16]:
merged_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,BCH-USD_close,BCH-USD_volume,ETH-USD_close,ETH-USD_volume
time,,,,,,,,
1528968720,6487.379883,7.706374,96.660004,314.387024,870.859985,26.856577,486.01001,26.019083
1528968780,6479.410156,3.088252,96.570000,77.129799,870.099976,1.124300,486.00000,8.449400
1528968840,6479.410156,1.404100,96.500000,7.216067,870.789978,1.749862,485.75000,26.994646
1528968900,6479.979980,0.753000,96.389999,524.539978,870.000000,1.680500,486.00000,77.355759
1528968960,6480.000000,1.490900,96.519997,16.991997,869.989990,1.669014,486.00000,7.503300


In [7]:
for c in merged_df.columns:
    print(c)

BTC-USD_close
BTC-USD_volume
LTC-USD_close
LTC-USD_volume
BCH-USD_close
BCH-USD_volume
ETH-USD_close
ETH-USD_volume


In [8]:
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
CRYPTO_TO_PREDICT = "LTC-USD" 

In [9]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [10]:
merged_df['future'] = merged_df[f"{CRYPTO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

In [11]:
merged_df['target'] = list(map(classify, merged_df[f"{CRYPTO_TO_PREDICT}_close"], 
                                        merged_df["future"]))

In [12]:
merged_df.head()


,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,BCH-USD_close,BCH-USD_volume,ETH-USD_close,ETH-USD_volume,future,target
time,,,,,,,,,,
1528968720,6487.379883,7.706374,96.660004,314.387024,870.859985,26.856577,486.01001,26.019083,96.389999,0
1528968780,6479.410156,3.088252,96.570000,77.129799,870.099976,1.124300,486.00000,8.449400,96.519997,0
1528968840,6479.410156,1.404100,96.500000,7.216067,870.789978,1.749862,485.75000,26.994646,96.440002,0
1528968900,6479.979980,0.753000,96.389999,524.539978,870.000000,1.680500,486.00000,77.355759,96.470001,1
1528968960,6480.000000,1.490900,96.519997,16.991997,869.989990,1.669014,486.00000,7.503300,96.400002,0


## NORMALIZING AND CREATING SEQUENCES


































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [17]:
times = sorted(merged_df.index.values)
last_5pct = times[-int(0.05*len(times))]

validation_df = merged_df[(merged_df.index >= last_5pct)] 
merged_df = merged_df[(merged_df.index < last_5pct)]

In [ ]:
from sklearn import preprocessing
import numpy as np 

def preprocessing_df(df):
    df = df.drop('future', axis=1)
    for col in df.columns:
        if col!="target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            
            df[col] = preprocessing.scale(df[col].values)
            
    df.dropna(inplace=True)
    
    seq_data = []
    previous_days = deque(maxlen=SEQ_LEN)
    
    for i in df.values:
        previous_days.append([n for n in i[:-1]])
        if len(previous_days) == SEQ_LEN:
            seq_data.append([np.array(previous_days), i[-1]])
            
    random.shuffle(seq_data)
    